Import packages, read data and drop unnecessary rows

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/kaggle/input/forest-surfaces-evolution-in-romania-19902019/forest_surfaces_romania_en_1990_2019.csv')
print(df.info())
# print(df.describe())
# columns -- Category / Region / Year / Unit ] Value

df.drop(df[df.Region.str.contains('Regiunea')].index, axis=0, inplace=True)
df.drop(df[df.Region.str.contains('MACROREGIUNEA')].index, axis=0, inplace=True)
df.drop(df[df.Region.str.contains('TOTAL')].index, axis=0, inplace=True)
print(df['Region'].nunique())
for col in df.columns:
    print(df[col].unique())

Check for incomplete data

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(df.isnull(), ax=ax)
plt.show()

In [ ]:
# VISUALISATION
# VISUALISATION
df_v= pd.DataFrame(df.groupby('Year')['Value'].sum())
df_v.plot.line(y='Value', figsize=(10,3),  color='green')
plt.title('Forest surfaces (K ha) evolution in Romania 1990-2019')
plt.show()

Create a pivot table region / year / values. Add the region id column

In [ ]:
df_counties = pd.read_csv('/kaggle/input/romania-counties/counties_id.csv')
df_counties = df_counties.drop(['date','populatie'], axis=1)
df = df.merge(df_counties[['Region','id']], on='Region', how='inner',)
pivot = df.pivot_table(index='Region', columns='Year', values='Value')
pivot['Evo'] = ((pivot[1990] - pivot[2019])*100/pivot[1990]).round(2)
pivot = pivot.merge(df_counties, on='Region', how='inner')
pivot.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pivot.drop(["a"], axis=1, inplace=True)
print(pivot.head(5))

Visualization of surface evolution

In [ ]:
import json
from plotly.offline import plot
import plotly.express as px

with open('/kaggle/input/romania-counties/romania.geojson.json', 'r') as response:
    counties = json.load(response)

fig = px.choropleth_mapbox(pivot,
                           geojson=counties,
                           featureidkey=str("properties.cartodb_id"),
                           locations='id',
                           color='Evo',
                           hover_name='Evo',
                           hover_data={"Evo": False, 'id':False, "Region": True},
                           color_continuous_scale="rdylgn",
                           color_continuous_midpoint=0,
                           range_color=(-30,30),
                           mapbox_style="carto-darkmatter",
                           zoom=5.5, center={"lat": 46, "lon": 25.5},
                           opacity=0.8,
                           labels={'Evo': 'Evolution %'},
                           # animation_frame='date',
                           )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0},
    annotations=[
        dict(
            text='Forest surface evolution for 1989-2019 - per counties',
            font=dict(
                family='sans-serif',
                size=15,
                color='#222'
            ),
            bgcolor='#E2E2E2',
            bordercolor='#FFFFFF',
            borderwidth=2,
            x=0.02,
            y=0.85,
            xanchor="left",
            yanchor="bottom"
        )])
fig.show()